# Actividad - Proyecto práctico

El proyecto ha sido desarrollado por:

*   **Alumno** 1: Jorge Aliaga 
*   **Alumno 2**: Lois Seijo
*   **Alumno 3**: Javier Berjoyo
*   **Alumno 4**: Leasly García

### Índice de contenidos

- [**1. Preparación del entorno**](#ap1)

- [**2. Space Invaders**](#ap2)

- [**3. Desarrollo del proyecto**](#ap3)
    - 3.1. [Implementación de la red neuronal](#ap3_1)
        - 3.1.1. [DQN](#ap_3_3_1)
        - 3.1.2. [Double DQN](#ap_3_3_2)
	    - 3.1.3. [Duel DQN](#ap_3_3_3)
       
    - 3.2. [Implementación de la solución DQN](#ap3_2)

    - 3.3. [Análisis](#ap3_3)

---
## 1. Preparación de entorno

Localizar entorno de trabajo: Google colab o local:

In [1]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount=''
drive_root = mount + ""

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

Montar carpeta de datos local (solo Colab):

In [2]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

Archivos en el directorio: 
['.git', '.gitignore', 'checkpoint', 'dqn2_SpaceInvaders-v0_log.json', 'dqn2_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn2_SpaceInvaders-v0_weights.h5f.index', 'dqn2_SpaceInvaders-v0_weights_250000.h5f.data-00000-of-00001', 'dqn2_SpaceInvaders-v0_weights_250000.h5f.index', 'dqn2_SpaceInvaders-v0_weights_500000.h5f.data-00000-of-00001', 'dqn2_SpaceInvaders-v0_weights_500000.h5f.index', 'dqn2_SpaceInvaders-v0_weights_750000.h5f.data-00000-of-00001', 'dqn2_SpaceInvaders-v0_weights_750000.h5f.index', 'dqn3_SpaceInvaders-v0_log.json', 'dqn3_SpaceInvaders-v0_weights.h5f.data-00000-of-00001', 'dqn3_SpaceInvaders-v0_weights.h5f.index', 'dqn3_SpaceInvaders-v0_weights_250000.h5f.data-00000-of-00001', 'dqn3_SpaceInvaders-v0_weights_250000.h5f.index', 'dqn3_SpaceInvaders-v0_weights_500000.h5f.data-00000-of-00001', 'dqn3_SpaceInvaders-v0_weights_500000.h5f.index', 'dqn3_SpaceInvaders-v0_weights_750000.h5f.data-00000-of-00001', 'dqn3_SpaceInvaders-v0_weights_75

Instalar librerías necesarias:

In [3]:
if IN_COLAB:
    %pip install gym==0.17.3
    %pip install git+https://github.com/Kojoley/atari-py.git
    %pip install keras-rl2==1.0.5
    %pip install tensorflow==2.8
else:
    %pip install gym==0.17.3
    %pip install git+https://github.com/Kojoley/atari-py.git
    %pip install pyglet==1.5.0
    %pip install h5py==3.1.0
    %pip install Pillow==9.5.0
    %pip install keras-rl2==1.0.5
    %pip install Keras==2.2.4
    %pip install tensorflow==2.5.3
    %pip install torch==2.0.1
    %pip install agents==1.4.0

Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
^C
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/Kojoley/atari-py.git to c:\users\javie\appdata\local\temp\pip-req-build-1iw2inzx
  Resolved https://github.com/Kojoley/atari-py.git to commit 86a1e05c0a95e9e6233c3a413521fdb34ca8a089
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/Kojoley/atari-py.git 'C:\Users\javie\AppData\Local\Temp\pip-req-build-1iw2inzx'


Note: you may need to restart the kernel to use updated packages.


---
## 2. Space Invaders


El objetivo de este proyecto es desarrollar un algoritmo, que mediante técnicas de aprendizaje por refuerzo, sea capaz de jugar en conocido juego **Space Invaders**. Para ello:

- El entorno de trabajo elegido será SpaceInvaders-v0.
- El algoritmo utilizado es DQN.

Este proyecto práctico consta de tres partes:

1. Implementar la red neuronal que se usará en la solución
2. Implementar las distintas piezas de la solución DQN
3. ustificar la respuesta en relación a los resultados obtenido

![Space Invaders][./imagenes/preprocessing.jpg]

---
## 3. Desarrollo del proyecto

#### Importar librerías

In [1]:
from __future__ import division

from PIL import Image
import numpy as np
import gym

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint

c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\Users\javie\anaconda3\envs\gym\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


#### Configuración base

En primer lugar, se crea el environment asociado al Space Invaders. Esto es posible gracias a la librería gymnasium. Tras ello, se asocia una aletoriedad al mismo. Por otro lado, se define el tamaño de la imagen de entrada y se obtienen las el número de accciones del entorno.

In [ ]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)

np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [ ]:
print("Numero de acciones disponibles: " + str(nb_actions))

In [ ]:
print("Formato de las observaciones:")
env.observation_space

Un espacio de soluciones de dimensiones (210, 160, 3) tiene un tamaño inmanejable. Se puede reducir ese espacio a un tamaño de 84 x 84 con una window length 4.

![Space Invaders](imagenes/space_invaders.gif)

Creación de la clase asociada al preprocesado:

In [ ]:
class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('uint8')

    def process_state_batch(self, batch):
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)

## 3.1. Implementación de la red neuronal

Antes de comentar que modelos se han utilizado, es necesario explicar como funciona el modelo básico de Deep Q Networks:

Para entender este modelo hace falta explicar una serie de conceptos:

### Recompesas a futuro y Discounted Rewards

En el aprendizaje por refuerzo, el objetivo del agente es maximizar la recompensa total a lo largo del tiempo. Sin embargo, no todas las recompensas son iguales. Las recompensas inmediatas suelen ser más valiosas que las recompensas futuras. Para darle un mayor peso a las recompensas más cercanas temporalmente, se introduce el concepto de recompensas descontadas/Discounted Rewards.

La recompensa total descontada en el tiempo t se define como:

Gt = ∑(γ^k * R(t+k))

Donde:

· Rt+k es la recompensa recibida en el tiempo t+k.
· γ es el factor de descuento, un valor entre 0 y 1. Un γ cercano a 1 indica que las recompensas futuras son casi tan valiosas como las inmediatas, mientras que un valor cercano a 0 indica que las recompensas inmediatas son mucho más importantes.

### Función Valor (Función V)

La función de valor/value, V(s), estima el valor esperado a largo plazo de un estado s. En otras palabras, representa la recompensa total descontada que se espera obtener comenzando desde el estado s y siguiendo una política específica π. Se entender como: "¿Cómo de bueno es el estado actual?"

V(s) = Eπ[Gt | st = s]

Donde:

Eπ denota la expectativa bajo la política π.
Gt es la recompensa total descontada, como se definió anteriormente.
st = s indica que el agente se encuentra en el estado s.

### Función Calidad (Función Q)

La función acción-valor, calidad/quality o función Q, Q(s, a), estima el valor esperado de tomar una acción específica a en un estado s y seguir la política π después.

Q(s, a) = Eπ[Gt | st = s, at = a]

Esta función representa la calidad de tomar una acción particular en un estado particular, en base a las recompensas futuras esperadas.


### Función de Pérdidas

En Deep Q-Networks, se utiliza una red neuronal para aproximar la función Q. La función de pérdida se define para minimizar la diferencia entre el valor Q estimado por la red y el valor Q objetivo calculado con una target network y una versión antigua de la red.

L(θ) = (y - Q(s, a; θ))^2

Donde:

y es el valor objetivo, que se puede escribir como Q(s', a'; θ').  Es el valor Q estimado para el siguiente estado s' usando una target network con parámetros antiguos θ'.

La minimización de esta función de pérdida ajusta, mediante descenso del gradiente, los parámetros de la red neuronal para que la estimación de Q(s, a; θ) se acerque al valor de y.

Además se han implementado las siguientes arquitecturas que suponen una mejoría con respecto a la versión estándar:

1.   Double DQN
2.   Duel DQN

Las funciones para la creación de las 3 arquitecturas se muestran a continuación:

### 3.1.1. DQN

In [ ]:
def construct_q_network(num_actions,dim=(4,84,84), loss='mse', optimizer=Adam(learning_rate=0.00001)):

    # Usamos la misma que en el paper de DeepMind salvo el permute
    model = Sequential()

    # Crear un nuevo modelo secuencial con la capa permutada
    model.add(Permute((3, 2, 1),input_shape=dim)) #IMPORTANTE 
    model.add(Convolution2D(32, (8, 8), strides=(4, 4)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (4, 4), strides=(2, 2)))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(num_actions))
    model.compile(loss=loss, optimizer=optimizer)

    print("Successfully constructed networks.")
    
    return model

### 3.1.2. Double DQN

In [ ]:
def construct_double_q_network(input_shape, num_actions, loss='mse', optimizer=Adam(lr=0.000001)):
       
        input_layer = Input(shape=input_shape)
        permuted_layer = Permute((3, 2, 1))(input_layer)   #IMPORTANTE 
        conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(permuted_layer)
        conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
        conv3 = Convolution2D(64, (3, 3), activation='relu')(conv2)
        flatten = Flatten()(conv3)

        fc1 = Dense(512)(flatten)
        advantage = Dense(num_actions)(fc1)

        fc2 = Dense(512)(flatten)
        value = Dense(1)(fc2)

        policy = concatenate([advantage, value], axis=-1)
        #policy = Lambda(lambda x: x[:, :-1] - K.mean(x[:, :-1], axis=1, keepdims=True) + x[:, -1])(policy)
        def custom_lambda(x):
            adv = x[:, :-1]
            val = x[:, -1:]
            print(f"adv shape: {adv.shape}, val shape: {val.shape}")
            return adv - K.mean(adv, axis=1, keepdims=True) + val

        policy = Lambda(custom_lambda)(policy)


        model = Model(inputs=input_layer, outputs=policy)
        model.compile(loss=loss, optimizer=optimizer)


        print("Successfully constructed networks.")

        return model

### 3.1.3. Duel DQN

In [ ]:
def construct_duel_q_network(input_shape, num_actions, loss='mse', optimizer=Adam(lr=0.000001)):
    input_layer = Input(shape=input_shape)
    print(K.image_data_format())
    if K.image_data_format() == 'channels_last':
    # (width, height, channels)
        permuted_layer=Permute((2, 3, 1), input_shape=input_shape)(input_layer)
    elif K.image_data_format() == 'channels_first':
    # (channels, width, height)
        permuted_layer=Permute((2, 3, 1), input_shape=input_shape)(input_layer)
    else:
        raise RuntimeError('Unknown image_dim_ordering.')
    conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(permuted_layer)
    conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
    conv3 = Convolution2D(64, (3, 3), activation='relu')(conv2)
    flatten = Flatten()(conv3)

    fc1 = Dense(512)(flatten)
    advantage = Dense(num_actions)(fc1)

    fc2 = Dense(512)(flatten)
    value = Dense(1)(fc2)

    policy = concatenate([advantage, value], axis=-1)
    policy = Lambda(lambda x: x[:, :-1] - K.mean(x[:, :-1], axis=1, keepdims=True) + x[:, -1])(policy)

    model = Model(inputs=input_layer, outputs=policy)
    model.compile(loss=loss, optimizer=optimizer)

    return model

## 3.2. Implementación de la solución DQN

Tamaño de la memoria para experience replay: 50000. Pensamos en utilizar un tamaño menor (10000), pero preferíamos tener un conjunto más grande y heterogéneo de experiencias. Usar más nos parecía excesivo porque estos entrenamientos ya eran suficientemente largos para un trabajo de estas dimensiones (18-24h).

Policy: se ha decidido usar la Linear Annealed Policy reduciendo la epsilon desde 1 hasta 0.1 (en distintas cantidades de steps). Se escoge porque consideramos que es necesaria cierta cantidad de exploración inicial en el experimento para conseguir una mejor generalización en test o que el agente descubra nuevas estrategias de acciones a seguir que previamente no se consideraban. Además, Tras un pequeño entrenamiento de prueba realizado al principio de nuestro estudio sobre el proyecto en el que usamos una EpsGreedyQPolicy, se podía ver que la nave repetía los mismos patrones o similares. Por esto se decidió cambiar la policy para promover la exploración.

In [ ]:
memory = SequentialMemory(limit=50000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy =  LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=450000)

In [ ]:
#Se creará una duel deep q network

model = construct_q_network(dim=(4,84,84), num_actions=6, loss='mse')

model.summary()

dqn = DQNAgent(model=model, 
                nb_actions=nb_actions,
                processor=processor,
                memory=memory,
                nb_steps_warmup=50000,
                target_model_update=1e-2,
                #enable_dueling_network=True, #Activamos el dueling para la duel deep q network 
                #dueling_type='avg', 
                policy=policy) 

dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

## 3.3. Pruebas de entrenamiento del modelo
Entrenamos durante 750000 steps y guardamos los pesos cada 250000. 

In [ ]:
# Training part
weights_filename = 'dqn_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(env, callbacks=callbacks, nb_steps=750000, log_interval=10000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

AQUI FALTARIA HABLAR DE QUE TODAS LAS PRUEBAS SSALIERON MAL . NO LLEGABAN A 20

---
## **PARTE 4**. Solución final

Finalmente, tras revisar las posibles soluciones al estancamiento del entrenamiento, se optó por crear un entorno que generara con cierta probabilidad las conocidas Sticky actions, las cuales ayudarían a mejorar la exploración, que se suponía uno de los principales problemas de nuestro entrenamiento.

Para ello, se sobreescribió el env básico para generar acciones pegajosas con cierta probabilidad:

In [2]:
class StickyActionEnv(gym.Wrapper):
    def __init__(self, env, sticky_prob=0.25):
        super(StickyActionEnv, self).__init__(env)
        self.sticky_prob = sticky_prob
        self.last_action = None

    def step(self, action):
        if self.last_action is not None and np.random.rand() < self.sticky_prob:
            action = self.last_action
        self.last_action = action
        obs, reward, done, info = self.env.step(action)
        return obs, reward, done, info

    def reset(self, **kwargs):
        self.last_action = None
        return self.env.reset(**kwargs)

#### Configuración base

Se mantuvo la misma configuración base, pero añadiendo el sticky env con una probabilidad de 0.25 de realizar sticky actions

In [3]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4
IN_COLAB=False

env_name = 'SpaceInvaders-v0'
env = gym.make(env_name)
sticky_env = StickyActionEnv(env, sticky_prob=0.25)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

In [4]:
print("Numero de acciones disponibles:" + str(nb_actions))
print("Formato de las observaciones:")
sticky_env.observation_space

Numero de acciones disponibles:6
Formato de las observaciones:


Box(0, 255, (210, 160, 3), uint8)

Como red neuronal se utilizó la misma double q network, por falta de tiempo no se pudo probar este sticky env con la dual q network, lo cual habría sido tambien interesante.

In [6]:
# arquitectura de DeepMind paper
def construct_double_q_network(input_shape, num_actions, loss='mse', optimizer=Adam(lr=0.0001)):

        input_layer = Input(shape=input_shape)
        permuted_layer = Permute((3, 2, 1))(input_layer)   #IMPORTANTE 
        conv1 = Convolution2D(32, (8, 8), strides=(4, 4), activation='relu')(permuted_layer)
        conv2 = Convolution2D(64, (4, 4), strides=(2, 2), activation='relu')(conv1)
        conv3 = Convolution2D(64, (3, 3), activation='relu')(conv2)
        flatten = Flatten()(conv3)

        fc1 = Dense(512)(flatten)
        advantage = Dense(num_actions)(fc1)

        fc2 = Dense(512)(flatten)
        value = Dense(1)(fc2)

        policy = concatenate([advantage, value], axis=-1)
        #policy = Lambda(lambda x: x[:, :-1] - K.mean(x[:, :-1], axis=1, keepdims=True) + x[:, -1])(policy)
        def custom_lambda(x):
            adv = x[:, :-1]
            val = x[:, -1:]
            print(f"adv shape: {adv.shape}, val shape: {val.shape}")
            return adv - K.mean(adv, axis=1, keepdims=True) + val

        policy = Lambda(custom_lambda)(policy)


        model = Model(inputs=input_layer, outputs=policy)
        model.compile(loss=loss, optimizer=optimizer)


        print("Successfully constructed networks.")

        return model

c:\Users\javie\anaconda3\envs\gym\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Misma configuracion de memoria y policy

In [10]:
memory = SequentialMemory(limit=50000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
# Definir la política base
base_policy = EpsGreedyQPolicy()

# Configurar LinearAnnealedPolicy con los parámetros necesarios
policy = LinearAnnealedPolicy(base_policy,
                              attr='eps',  # El atributo que se debe anillar
                              value_max=1.0,  # Valor inicial de epsilon
                              value_min=0.1,  # Valor final de epsilon
                              value_test=0.05,  # Valor de epsilon durante la prueba
                              nb_steps=10000)  # Número de pasos para el anillado

In [12]:
model = construct_double_q_network(input_shape=(4, 84,84), num_actions=6, loss='mse', optimizer=Adam(lr=0.0001))
model.summary()


adv shape: (None, 6), val shape: (None, 1)
Successfully constructed networks.
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 4, 84, 84)]  0                                            
__________________________________________________________________________________________________
permute_2 (Permute)             (None, 84, 84, 4)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 20, 20, 32)   8224        permute_2[0][0]                  
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 9, 9, 64)     32832       conv2d_6[0][0]                   
______________

Tambien misma configuracion de DQNAgent

In [9]:
dqn = DQNAgent(model=model, 
                nb_actions=nb_actions,
                processor=processor,
                memory=memory,
                nb_steps_warmup=50000,
                target_model_update=1e-2, 
                policy=policy, 
                enable_double_dqn=True) #target model te congela durante 100 steps lo que se usa para el "ground truth"

dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])

adv shape: (None, 6), val shape: (None, 1)


Entrenamiento 

In [13]:
# Training part
weights_filename = 'dqn3_{}_weights.h5f'.format(env_name)
checkpoint_weights_filename = 'dqn3_' + env_name + '_weights_{step}.h5f'
log_filename = 'dqn3_{}_log.json'.format(env_name)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=250000)]
callbacks += [FileLogger(log_filename, interval=100)]

dqn.fit(sticky_env, callbacks=callbacks, nb_steps=1000000, log_interval=10000, visualize=False)

dqn.save_weights(weights_filename, overwrite=True)

Training for 1000000 steps ...
Interval 1 (0 steps performed)


c:\Users\javie\anaconda3\envs\gym\lib\site-packages\tensorflow\python\keras\engine\training.py:2424: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


10000/10000 [==============================] - 76s 8ms/step - reward: 0.0119: 0s - reward: 
13 episodes - episode_reward: 8.769 [5.000, 13.000] - ale.lives: 2.164

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 74s 7ms/step - reward: 0.0147
11 episodes - episode_reward: 13.818 [11.000, 20.000] - ale.lives: 1.979

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: 0.0142
10 episodes - episode_reward: 13.000 [6.000, 16.000] - ale.lives: 2.109

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 77s 8ms/step - reward: 0.0133
12 episodes - episode_reward: 11.667 [11.000, 15.000] - ale.lives: 2.110

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 66s 7ms/step - reward: 0.0148
11 episodes - episode_reward: 13.000 [10.000, 17.000] - ale.lives: 2.095

Interval 6 (50000 steps performed)
10000/10000 [==============================] - 1522s 152ms

Por falta de tiempo nos vimos obligados a parar en 750000 steps

In [15]:
# Testing part to calculate the mean reward
weights_filename = 'dqn3_{}_weights_750000.h5f'.format(env_name)
dqn.load_weights(weights_filename)
dqn.test(env, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 19.000, steps: 1074
Episode 2: reward: 24.000, steps: 831
Episode 3: reward: 26.000, steps: 1223
Episode 4: reward: 22.000, steps: 890
Episode 5: reward: 13.000, steps: 629
Episode 6: reward: 29.000, steps: 1231
Episode 7: reward: 14.000, steps: 666
Episode 8: reward: 17.000, steps: 703
Episode 9: reward: 18.000, steps: 971
Episode 10: reward: 29.000, steps: 1141


El resultado es una media de 22 de recompensa, lo cual cumple con los objetivos.

---